In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from qutip import Bloch
from scipy.optimize import minimize
from functools import reduce
import itertools
from ipywidgets import interact, interactive_output, IntSlider, FloatSlider, VBox, HBox, Button, Output, Label
import ipywidgets as widgets
from IPython.display import display, clear_output
class QuantumStateTomography:
    """Class for quantum state tomography and visualization."""
    
    def __init__(self, n_qubits, n_shotsX=1000, n_shotsY=1000, n_shotsZ=1000):
        """
        Initialize the quantum state tomography class.
        
        Args:
            n_qubits (int): Number of qubits
            n_shotsX (int): Number of measurement shots for X basis
            n_shotsY (int): Number of measurement shots for Y basis
            n_shotsZ (int): Number of measurement shots for Z basis
        """
        self.n_qubits = n_qubits
        self.n_shotsX = n_shotsX
        self.n_shotsY = n_shotsY
        self.n_shotsZ = n_shotsZ
        
        # Pauli matrices dictionary
        self.pauli_dict = {
            'I': np.array([[1,0],[0,1]]),
            'X': np.array([[0,1],[1,0]]),
            'Y': np.array([[0,-1j],[1j,0]]),
            'Z': np.array([[1,0],[0,-1]])
        }
        self.pauli_letters = ['I', 'X', 'Y', 'Z']
        
        # Initialize true parameters
        self.true_thetas = [np.random.uniform(0, np.pi) for _ in range(2**n_qubits-1)]
        self.true_phis = [np.random.uniform(0, 2*np.pi) for _ in range(2**n_qubits-1)]
        self.true_params = np.concatenate([self.true_thetas, self.true_phis])
        self.n_theta_phi = len(self.true_thetas)
        
        # Generate true state
        self.true_state = self.generate_multi_qubit_state(self.true_thetas, self.true_phis)
        
    def generate_multi_qubit_state(self, thetas, phis):
        """
        Generate a multi-qubit pure state in polar form.
        
        Args:
            thetas: List of theta angles (in radians)
            phis: List of phi angles (in radians)
            
        Returns:
            state: Numpy array of shape (2**n_qubits,) representing the quantum state
        """
        if len(thetas) != 2**self.n_qubits - 1:
            raise ValueError(f"Expected {2**self.n_qubits - 1} theta parameters, got {len(thetas)}.")
        if len(phis) != 2**self.n_qubits - 1:
            raise ValueError(f"Expected {2**self.n_qubits - 1} phi parameters, got {len(phis)}.")

        dim = 2**self.n_qubits
        state = np.zeros(dim, dtype=complex)

        amplitude = 1.0
        for idx in range(dim-1):
            state[idx] = amplitude * np.cos(thetas[idx]/2)
            amplitude *= np.sin(thetas[idx]/2)
            if idx > 0:
                state[idx] *= np.exp(1j * phis[idx-1])

        state[idx+1] = amplitude * np.exp(1j * phis[idx])
        return state

    def meas_prob(self, state, meas_op_string):
        """
        Calculate measurement probability for given operator string.
        
        Args:
            state: Quantum state vector
            meas_op_string: String of Pauli operators (e.g., 'XX')
            
        Returns:
            prob_plus1: Probability of +1 outcome
        """
        dim = len(state)
        meas_op = [self.pauli_dict[op] for op in meas_op_string]
        M = reduce(np.kron, meas_op)
        proj_M = (np.eye(dim) + M)/2
        prob_plus1 = (np.conj(state).T) @ proj_M @ state
        return prob_plus1.real

    def generate_samples(self, state, meas_op_string):
        """
        Simulate measurements in the given basis.
        
        Args:
            state: Quantum state vector
            meas_op_string: String of Pauli operators
            
        Returns:
            samples: Array of measurement outcomes (+1 or -1)
        """
        shots_list = []
        for q_idx in range(self.n_qubits):
            if meas_op_string[q_idx] == 'X':
                shots_list.append(self.n_shotsX)
            elif meas_op_string[q_idx] == 'Y':
                shots_list.append(self.n_shotsY)
            elif meas_op_string[q_idx] == 'Z':
                shots_list.append(self.n_shotsZ)

        n_shots = min(shots_list)
        prob_plus1 = self.meas_prob(state, meas_op_string)
        prob_minus1 = 1 - prob_plus1
        return np.random.choice([+1, -1], size=n_shots, p=[prob_plus1, prob_minus1])

    def log_likelihood(self, params, samples):
        """
        Compute negative log-likelihood.
        
        Args:
            params: Concatenated theta and phi parameters
            samples: Dictionary of measurement outcomes
            
        Returns:
            Negative log-likelihood value
        """
        thetas = params[:self.n_theta_phi]
        phis = params[self.n_theta_phi:]
        state = self.generate_multi_qubit_state(thetas, phis)
        
        log_L = 0
        for setting in itertools.product(self.pauli_letters, repeat=self.n_qubits):
            if setting != ('I',) * self.n_qubits:
                prob_plus1 = self.meas_prob(state, setting)
                n_plus1 = np.sum(samples[setting] == +1)
                n_minus1 = np.sum(samples[setting] == -1)
                log_L += n_plus1 * np.log(prob_plus1 + 1e-10) + n_minus1 * np.log(1 - prob_plus1 + 1e-10)
        
        return -log_L

    def estimate_parameters(self, samples):
        """
        Estimate parameters using Maximum Likelihood Estimation.
        
        Args:
            samples: Dictionary of measurement outcomes
            
        Returns:
            Estimated parameters (thetas and phis concatenated)
        """
        opt_val = []
        opt_params = []
        
        for _ in range(50):
            theta_init = np.random.uniform(0, np.pi, size=2**self.n_qubits-1)
            phi_init = np.random.uniform(0, 2*np.pi, size=2**self.n_qubits-1)
            initial_params = np.concatenate([theta_init, phi_init])
            
            bounds = [(0, np.pi)] * (2**self.n_qubits-1) + [(0, 2*np.pi-0.01)] * (2**self.n_qubits-1)
            
            result = minimize(self.log_likelihood, initial_params, args=(samples),
                            bounds=bounds, method='L-BFGS-B')
            opt_val.append(result.fun)
            opt_params.append(result.x)
        
        result = min(opt_val)
        index = opt_val.index(result)
        return opt_params[index]

    def fidelity(self, state1, state2):
        """
        Compute fidelity between two quantum states.
        
        Args:
            state1, state2: Quantum state vectors
            
        Returns:
            Fidelity value
        """
        return np.abs(np.vdot(state1, state2))**2

    def partial_trace_density(self, rho, target_qubit):
        """
        Compute partial trace to obtain reduced density matrix for target qubit.
        
        Args:
            rho: Full density matrix
            target_qubit: Qubit index to keep
            
        Returns:
            Reduced density matrix (2x2)
        """
        dim = 2**self.n_qubits
        reduced_rho = np.zeros((2,2), dtype=complex)

        for i in range(dim):
            for j in range(dim):
                i_bits = [(i >> k) & 1 for k in range(self.n_qubits)][::-1]
                j_bits = [(j >> k) & 1 for k in range(self.n_qubits)][::-1]
                if all(i_bits[k] == j_bits[k] for k in range(self.n_qubits) if k != target_qubit):
                    reduced_rho[i_bits[target_qubit], j_bits[target_qubit]] += rho[i,j]
        return reduced_rho

    def bloch_vector_from_rho(self, rho):
        """
        Compute Bloch vector from 2x2 density matrix.
        
        Args:
            rho: 2x2 density matrix
            
        Returns:
            Bloch vector (x, y, z)
        """
        x = 2 * np.real(rho[0,1])
        y = 2 * np.imag(rho[1,0])
        z = np.real(rho[0,0] - rho[1,1])
        return np.array([x, y, z])

    def plot_multi_bloch(self, input_state, title="Bloch Spheres"):
        """
        Plot Bloch spheres using QuTiP Bloch class.
        
        Args:
            input_state: State vector or density matrix
            title: Plot title
        """
        if len(input_state.shape) == 1:
            rho = np.outer(input_state, input_state.conj())
        else:
            rho = input_state
    
        fig, axes = plt.subplots(1, self.n_qubits, figsize=(4*self.n_qubits, 4), 
                                 subplot_kw={'projection': '3d'})
        
        if self.n_qubits == 1:
            axes = [axes]
    
        for qubit in range(self.n_qubits):
            reduced_rho = self.partial_trace_density(rho, qubit)
            bloch_vec = self.bloch_vector_from_rho(reduced_rho)
            
            b = Bloch(axes=axes[qubit])
            b.add_vectors(bloch_vec)
            b.render()
            axes[qubit].set_title(f"Qubit {qubit}")
    
        fig.suptitle(title, fontsize=16)
        plt.tight_layout()
        plt.show()  # Ensure the plot is displayed in Jupyter
    
    def plot_thetas_phis(self, true_thetas, true_phis, est_thetas, est_phis, fidelity):
   
        n_params = len(true_thetas)
        fig = plt.figure(figsize=(max(10, n_params*1.5), 5))
        gs = fig.add_gridspec(2, 1, height_ratios=[1, 4], hspace=0.4)
        
        # Fidelity subplot
        ax_fid = fig.add_subplot(gs[0])
        ax_fid.set_xlim(0, 1)
        ax_fid.set_ylim(0, 1)
        ax_fid.axis('off')
        bg_bar = patches.Rectangle((0, 0.4), 1, 0.2, color='lightgray', ec='lightgray')
        fid_bar = patches.Rectangle((0, 0.4), fidelity, 0.2, color='red')
        ax_fid.add_patch(bg_bar)
        ax_fid.add_patch(fid_bar)
        ax_fid.text(0.5, 0.8, f"Fidelity: {fidelity:.4f}", ha='center', fontsize=16)
        
        # Parameters subplot
        ax = fig.add_subplot(gs[1])
        labels, values, colors = [], [], []
        for i in range(n_params):
            labels.append(r'$\theta_{{{}}}^{{\text{{True}}}}$'.format(i))
            labels.append(r'$\theta_{{{}}}^{{\text{{Est}}}}$'.format(i))
            values.append(true_thetas[i])
            values.append(est_thetas[i])
            colors.append('green')
            colors.append('lightgreen')
        for i in range(n_params):
            labels.append(r'$\phi_{{{}}}^{{\text{{True}}}}$'.format(i))
            labels.append(r'$\phi_{{{}}}^{{\text{{Est}}}}$'.format(i))
            values.append(true_phis[i])
            values.append(est_phis[i])
            colors.append('blue')
            colors.append('lightblue')
        
        x_pos = np.arange(len(labels))
        ax.bar(x_pos, values, color=colors)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(labels, rotation=90, fontsize=10)  # Rotate labels to 90 degrees and adjust fontsize
        yticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
        ytick_labels = ['0', r'$\pi/2$', r'$\pi$', r'$3\pi/2$', r'$2\pi$']
        ax.set_yticks(yticks)
        ax.set_yticklabels(ytick_labels, fontsize=13)
        ax.set_ylim(0, 2*np.pi + 0.75)
        ax.set_ylabel('Angle (radians)', fontsize=14)
        ax.set_title('True and Estimated θ, φ Parameters', fontsize=16)
        for i, val in enumerate(values):
            ax.text(i, val + 0.1, f"{np.degrees(val):.1f}°", ha='center', va='bottom', fontsize=11,rotation=90)
        
        plt.tight_layout()  # Adjust layout to prevent clipping
        plt.show()

    def run_tomography(self):
        """
        Run complete quantum state tomography process.
        
        Returns:
            Dictionary containing true and estimated parameters and fidelity
        """
        # Generate samples
        samples = {}
        for setting in itertools.product(self.pauli_letters, repeat=self.n_qubits):
            if setting != ('I',) * self.n_qubits:
                samples[setting] = self.generate_samples(self.true_state, setting)

        # Estimate parameters
        est_params = self.estimate_parameters(samples)
        est_thetas = est_params[:self.n_theta_phi]
        est_phis = est_params[self.n_theta_phi:]
        
        # Generate reconstructed state
        reconstructed_state = self.generate_multi_qubit_state(est_thetas, est_phis)
        
        # Calculate fidelity
        fid = self.fidelity(self.true_state, reconstructed_state)
        
        # Plot results
        self.plot_thetas_phis(self.true_thetas, self.true_phis, est_thetas, est_phis, fid)
        self.plot_multi_bloch(self.true_state, "True State")
        self.plot_multi_bloch(reconstructed_state, "Reconstructed State")
        return {
            'true_params': self.true_params,
            'estimated_params': est_params,
            'fidelity': fid,
            'reconstructed_state': reconstructed_state,  # Add reconstructed state to the return dictionary
            'estimated_thetas': est_thetas,  # Include estimated thetas
            'estimated_phis': est_phis  # Include estimated phis
        }
#if __name__ == "__main__":
    # Example usage
    #tomography = QuantumStateTomography(n_qubits=2)
    #results = tomography.run_tomography()
    #print("True parameters:", results['true_params'])
    #print("Estimated parameters:", results['estimated_params'])
    #print("Fidelity:", results['fidelity'])
 

# Input widgets
n_qubits_slider = IntSlider(value=2, min=1, max=4, step=1, description='Qubits:')
shotsX_slider = IntSlider(value=1000, min=100, max=5000, step=100, description='Shots X:')
shotsY_slider = IntSlider(value=1000, min=100, max=5000, step=100, description='Shots Y:')
shotsZ_slider = IntSlider(value=1000, min=100, max=5000, step=100, description='Shots Z:')
generate_angles_button = Button(description='Set θ and φ', button_style='info')
run_button = Button(description='Run Experiment', button_style='success')
output_box = Output()

# Placeholders for dynamically created theta and phi sliders
theta_sliders = []
phi_sliders = []
angle_box = VBox([])

def create_theta_phi_sliders(n_qubits):
    n_angles = 2**n_qubits - 1
    global theta_sliders, phi_sliders
    theta_sliders = [FloatSlider(value=np.pi/4, min=0, max=np.pi, step=0.01,
                                 description=f'θ{i}') for i in range(n_angles)]
    phi_sliders = [FloatSlider(value=np.pi, min=0, max=2*np.pi, step=0.01,
                               description=f'φ{i}') for i in range(n_angles)]
    angle_box.children = [Label("Set θ angles:")] + theta_sliders + [Label("Set φ angles:")] + phi_sliders

def on_generate_angles_clicked(b):
    create_theta_phi_sliders(n_qubits_slider.value)
    output_box.clear_output()
    with output_box:
        print(f"Generated {len(theta_sliders)} θ sliders and {len(phi_sliders)} φ sliders.")

generate_angles_button.on_click(on_generate_angles_clicked)
def on_run_button_clicked(b):
    output_box.clear_output()
    with output_box:
        n_qubits = n_qubits_slider.value
        n_shotsX = shotsX_slider.value
        n_shotsY = shotsY_slider.value
        n_shotsZ = shotsZ_slider.value

        true_thetas = [slider.value for slider in theta_sliders]
        true_phis = [slider.value for slider in phi_sliders]
        
        print(f"\nRunning tomography for {n_qubits} qubits...")
        print(f"Theta values ({len(true_thetas)}):", np.round(true_thetas, 3))
        print(f"Phi values   ({len(true_phis)}):", np.round(true_phis, 3))

        if len(true_thetas) != 2**n_qubits - 1 or len(true_phis) != 2**n_qubits - 1:
            print("Error: Number of angles does not match 2^n - 1")
            return

        qtomo = QuantumStateTomography(n_qubits=n_qubits,
                                       n_shotsX=n_shotsX,
                                       n_shotsY=n_shotsY,
                                       n_shotsZ=n_shotsZ)

        qtomo.true_thetas = true_thetas
        qtomo.true_phis = true_phis
        qtomo.true_params = np.concatenate([true_thetas, true_phis])
        qtomo.true_state = qtomo.generate_multi_qubit_state(true_thetas, true_phis)

        results = qtomo.run_tomography()
        print("\n✅ Tomography complete.")
        print(f"Fidelity: {results['fidelity']:.6f}")

        #print("\nBloch spheres for true and reconstructed states:")
       # qtomo.plot_multi_bloch(qtomo.true_state, title='True State Bloch Spheres')
        

In [9]:

run_button.on_click(on_run_button_clicked)

# Layout
config_box = VBox([n_qubits_slider, shotsX_slider, shotsY_slider, shotsZ_slider, generate_angles_button])
main_box = VBox([config_box, angle_box, run_button, output_box])
display(main_box)